<a href="https://colab.research.google.com/github/Dimildizio/Adjuface/blob/main/Block_1/nlp_intro_part2_hw_solved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введение в NLP, часть 2


In [1]:
!pip install -U pip
!pip install transformers datasets torch seqeval evaluate

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 71.9 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=cc54be1e502cf7544ab75440ddc6b0548fb3723d50b00c69957b430be54063cf
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installe


## NER c BERT (25 баллов)

1. Взять датасет из предыдущего ДЗ и обучить на нём BERT.
2. Обучить BERT на подготовленном датасете
3. Оценить результат, сравнить с моделью из первого ДЗ

### Подготовка данных (5 баллов)

Подумать о:
1) Как subword токенизация повлияет на BIO раззметку?
2) Что делать с `[CLS]` и `[SEP]` токенами? (Проверьте что использует `DataCollatorForTokenClassification`)

> Hint! Токенайзер умеет работать с предразделёнными на "слова" текстами

1. Subword токенизацию можно использовать для NER задач с BIO разметкой. Но поскольку у нас теперь только части слов, нам необходимо заново делать разметку добавляя больше `I-` меток. Однако, с уменьшением размера подслова будут чаще встречаться подслова которые входят в разные группы. Например `Санкт Петербург`
 - `B-LOC I-LOC` может стать:

 - `сан` - `B-LOC`

 - `кт` - `I-LOC`

 - `петер` - `I-LOC`

 - `бург` - `I-LOC`

 В то же время слово гамбургер может стать:

 - `гам` - `O`

 - `бург`  - `O`

 - `ер`  - `O`

 а Джордан Питерсан

 - `джордан` - `B-PER`

 - `петер` - `I-PER`

  - `сан` - `I-PER`

  Но это потенциально, зачастую можно предположить что метки `О` не надо разбивать

 2. [CLS] [SEP] токены можно игонрировать и как и в трансформеровском `DataCollatorForTokenClassification` можно присваивать значение `-100`

Хаггингфейсовыйтокенайзер умеет работать со словами - `is_split_into_words=True` что говорит ему о том что инпут подан в виде разбитых слов, а не предложений\текста, таким образом он через идентификаторы слов знает, какие токены соответствуют одному и тому же слову и можно распределить BIO-разметку не задумываясь о проблемах описанных выше.

В целом мне кажется что при учитывании этой специфики, такое разбитие на субслова должно помочь лучше справиться со сложными и сложносоставными именованными сущностями и может вполне помочь улучшить эффективность и метрики.


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer


BASE_NER_MODEL = "bert-base-cased"
bert_tokenizer = AutoTokenizer.from_pretrained(BASE_NER_MODEL)

conll2003 = load_dataset("conll2003")
conll2003

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:81: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

...

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [3]:
example = conll2003["train"][100]
example

{'id': '100',
 'tokens': ['Rabinovich',
  'is',
  'winding',
  'up',
  'his',
  'term',
  'as',
  'ambassador',
  '.'],
 'pos_tags': [21, 42, 39, 33, 29, 21, 15, 21, 7],
 'chunk_tags': [11, 21, 22, 15, 11, 12, 13, 11, 0],
 'ner_tags': [1, 0, 0, 0, 0, 0, 0, 0, 0]}

* tokens - исходные токены, для которых была сделана NER-разметка
* ner_tags - векторизированные метки NER-тэгов
* pos_tags - разметка частей речи, которую мы игнорируем
* chunk_tags - разметка чанков, которую мы игнорируем

Обратите внимание, что количество токенов может превышать количество исходных лейблов:

In [4]:
bert_tokenizer(example["tokens"], is_split_into_words=True).tokens()

['[CLS]',
 'Ra',
 '##bino',
 '##vich',
 'is',
 'winding',
 'up',
 'his',
 'term',
 'as',
 'ambassador',
 '.',
 '[SEP]']

Значение тэга в ner_tags отображается в метку NER:

In [5]:
print("NER TAGS", example["ner_tags"])
print(conll2003["train"].features["ner_tags"].feature)

NER TAGS [1, 0, 0, 0, 0, 0, 0, 0, 0]
ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None)


In [6]:
print("Оригинальные токены")
print(example["tokens"])
print("Векторизированные NER метки токенов")
print(example["ner_tags"])
tags_str = []
features = conll2003["train"].features["ner_tags"].feature
for tag in example["ner_tags"]:
    tags_str.append(features.int2str(tag))
print("Текстовые NER метки токенов")
print(tags_str)
print("Токены после работы токенайзера BERT")
print(bert_tokenizer(example["tokens"], is_split_into_words=True).tokens())

Оригинальные токены
['Rabinovich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.']
Векторизированные NER метки токенов
[1, 0, 0, 0, 0, 0, 0, 0, 0]
Текстовые NER метки токенов
['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Токены после работы токенайзера BERT
['[CLS]', 'Ra', '##bino', '##vich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.', '[SEP]']


Вспомним немного, как работают метки в задаче мер в кодировке BIO. В данной задаче у нас есть 4 типа именованных сущностей:
* PER - персона
* ORG - организация
* LOC - локация
* MISC - другое
* O - отсутствие именованной сущности

У каждого типа именованных 2 префикса:
* `B-` - beginning, т.е. начало именованной сущности.
* `I-` - inside, т.е. продолжение ранее начатой именованной сущностью.

В исходной токенизации

`['Rabinovich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.']`
метки выглядят как

`['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']`
т.е. `Rabinovich` является персоной. На следующем токене именованная сущность заканчивается, т.к. у него метка `O`.

После токенизации BERT наш сэмпл превращается в следующие токены:

`['[CLS]', 'Ra', '##bino', '##vich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.', '[SEP]']`
Обратим внимание, что один токен `Rabinovich` с меткой `B-PER` был разбит токенизатором берта на 3 токена: `'Ra', '##bino', '##vich'`. Им нужно поставить в соответствие 3 метки: `B-PER, I-PER, I-PER`, т.е. мы разбиваем метку исходного токена на новые токены.

Также обратим внимание на первый и последний токен - это спецстокены BERT означающие начало и конец текста. Им можно дать метки `O`, т.к. они не являются частью исходного текста, но мы будем давать им особое векторизированное значение -100. В [документации pytroch](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) у кроссэнтропийной функции потерь это дефолтное значение `ignore_index`, т.е. метки, которую мы будем игнорировать. Библиотека transformers также использует это значение. Таким образом на токенах, у которых стоит -100 в качестве векторизированного NER-тэга, не будет происходить обучение, они будут проигнорированы.




Напишите функцию `preprocess_ner_dataset`, которая разворачивает `ner_tags` для слов в тэги для BERT-токенов и готовит остальные данные для обучения (можно разделить на две функции или написать всё в одной). В резултате применения `conll2003.map(preprocess_ner_dataset)`, в каждом примере:
1. Добавляется токенизированный вход (`input_ids`, `token_type_ids` и `attention_mask`). При конструировании этих векторов вручную нужно проставить `attention_mask` полностью единицами, т.к. в паддинги в сэмплах появляются только в рамках батчей, а `token_type_ids` полностью нулями.
2. `ner_tags` разворачивается в `labels` для входных токенов

Что можно использовать:
* у объекта `conll2003["train"].features["ner_tags"].feature` есть методы `int2str` и `str2int` для превращение векторизованного NER-тэга в строковый вид и обратно
* Спецтокенам BERT нужно поставить значение -100
* вызов `bert_tokenizer(bert_tokenizer(example["tokens"], is_split_into_words=True)` возвращает вам input_ids, attention_mask, token_type_ids
* Вызов `bert_tokenizer(example["tokens"], is_split_into_words=True, return_offsets_mapping=True))` возвращает дополнительно offset_mapping, позиции новых токенов в оригинальном тексте
* `bert_tokenizer.vocab` - для превращения токенов в их индексы в словаре
* `bert_tokenizer.tokenize` - разбитие текста (в том числе и исходных токенов) на токены BERT

Ваша задача:
1. Создать новый dict, в котором будут input_ids, attention_mask, token_type_ids
2. Добавить в него labels - векторизированные NER-тэги, которые будут разбиты в соответствии с токенизацией BERT. Для этого можно можно разбить каждый токен отдельно и размножить его метки. Альтернативно можно использовать информацию об оффсетах токенов BERT, чтобы понять, частью какого исходного токена и какой исходной метки является данный BERT-токен.

In [25]:
def preprocess_ner_dataset(example):

    tokenized_inputs = bert_tokenizer(example["tokens"],
                                      is_split_into_words=True,
                                      return_offsets_mapping=True,  # Return offsets to map initial tokens
                                      padding=False,
                                      truncation=True)
    labels = []
    input_ids = tokenized_inputs["input_ids"]
    attention_mask = tokenized_inputs["attention_mask"]
    token_type_ids = tokenized_inputs["token_type_ids"]
    offset_mapping = tokenized_inputs["offset_mapping"]  # offset mapping to match tokens and their orig pos

    current_word_idx = 0  # Еo map tokens and дфиуды
    previous_word_idx = -1  # To track if current token is a part of a previous entity

    for idx, offset in enumerate(offset_mapping):
        if offset == (0, 0):  # if [CLS] or [SEP] se set -100
            labels.append(-100)
        else:
            # If curr token is a new word
            word_idx = tokenized_inputs.word_ids(batch_index=0)[idx]
            if word_idx != previous_word_idx:  # set label to original token
                label = example['ner_tags'][word_idx]
                labels.append(label)
                previous_word_idx = word_idx
            else:
                labels.append(-100)  # Set -100 to any further subtokens

    return {"input_ids": input_ids, "token_type_ids": token_type_ids, "attention_mask": attention_mask, "labels": labels}

{'input_ids': [101, 16890, 25473, 11690, 1110, 14042, 1146, 1117, 1858, 1112, 9088, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 1, -100, -100, 0, 0, 0, 0, 0, 0, 0, 0, -100]}


In [52]:
def preprocess_ner_dataset(example):
    # Токенизируем с разбиением на слова и возвращением оффсетов
    tokenized_inputs = bert_tokenizer(example["tokens"],
                                      is_split_into_words=True,
                                      return_offsets_mapping=True,  # Возвращаем оффсеты для маппинга исходных токенов
                                      padding=False,
                                      truncation=True)

    # Инициализация списка для меток
    labels = []
    input_ids = tokenized_inputs["input_ids"]
    attention_mask = tokenized_inputs["attention_mask"]
    token_type_ids = tokenized_inputs["token_type_ids"]
    offset_mapping = tokenized_inputs["offset_mapping"]  # Оффсеты для маппинга токенов и их оригинальных позиций

    previous_word_idx = None  # Для отслеживания предыдущего индекса слова

    for idx, offset in enumerate(offset_mapping):
        if offset == (0, 0):  # Спецтокены [CLS] и [SEP], ставим метку -100
            labels.append(-100)
        else:
            # Индекс текущего слова
            word_idx = tokenized_inputs.word_ids(batch_index=0)[idx]

            # Если токен является новым словом
            if word_idx != previous_word_idx:
                # Получаем метку для текущего слова
                label = example['ner_tags'][word_idx]
                labels.append(label)
            else:
                # Для субтокенов внутри того же слова используем метку I (например, если это "B-PER", ставим "I-PER")
                if labels[-1] != -100:  # Проверяем, что предыдущая метка была не для спецтокена
                    labels.append(2)  # Метка для I- (вместо размножения можно присвоить 2 для simplicity)
                else:
                    labels.append(-100)

            # Обновляем индекс предыдущего слова
            previous_word_idx = word_idx

    # Возвращаем оригинальные и новые данные
    return {
        "input_ids": input_ids,  # Токенизированные входные данные
        "token_type_ids": token_type_ids,  # Типы токенов
        "attention_mask": attention_mask,  # Маска внимания
        "labels": labels  # Развернутые NER метки для BERT-токенов
    }


example = conll2003["train"][200]
processed_example = preprocess_ner_dataset(example)
print(processed_example)

{'input_ids': [101, 157, 11612, 15824, 1592, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [-100, 3, 2, 2, 2, -100]}


In [55]:
def preprocess_ner_dataset(example):
    # Токенизируем с разбиением на слова и возвращением оффсетов
    tokenized_inputs = bert_tokenizer(example["tokens"],
                                      is_split_into_words=True,
                                      return_offsets_mapping=True,  # Возвращаем оффсеты для маппинга исходных токенов
                                      padding=False,
                                      truncation=True)

    # Инициализация списка для меток
    labels = []
    input_ids = tokenized_inputs["input_ids"]
    attention_mask = tokenized_inputs["attention_mask"]
    token_type_ids = tokenized_inputs["token_type_ids"]
    offset_mapping = tokenized_inputs["offset_mapping"]  # Оффсеты для маппинга токенов и их оригинальных позиций

    previous_word_idx = None  # Для отслеживания предыдущего индекса слова

    for idx, offset in enumerate(offset_mapping):
        if offset == (0, 0):  # Спецтокены [CLS] и [SEP], ставим метку -100
            labels.append(-100)
        else:
            # Индекс текущего слова
            word_idx = tokenized_inputs.word_ids(batch_index=0)[idx]

            # Если токен является новым словом
            if word_idx != previous_word_idx:
                # Получаем метку для текущего слова
                label = example['ner_tags'][word_idx]
                labels.append(label)
                previous_label = label  # Сохраняем текущую метку, чтобы размножить её на субтокены
            else:
                # Для субтокенов внутри того же слова используем метку I соответствующей сущности
                if labels[-1] != -100:  # Проверяем, что предыдущая метка была не для спецтокена
                    if previous_label % 2 == 1:  # Если метка начинается с B (например, B-ORG = 3)
                        labels.append(previous_label + 1)  # Присваиваем метку I (например, I-ORG = 4)
                    else:
                        labels.append(previous_label)  # Если уже I, остаётся I
                else:
                    labels.append(-100)

            # Обновляем индекс предыдущего слова
            previous_word_idx = word_idx

    # Возвращаем оригинальные и новые данные
    return {
        "input_ids": input_ids,  # Токенизированные входные данные
        "token_type_ids": token_type_ids,  # Типы токенов
        "attention_mask": attention_mask,  # Маска внимания
        "labels": labels  # Развернутые NER метки для BERT-токенов
    }

Пример получившегося выхода:
```python
>>> preprocessed_ner_dataset["train"][100]
{'id': '100',
 'tokens': ['Rabinovich',
  'is',
  'winding',
  'up',
  'his',
  'term',
  'as',
  'ambassador',
  '.'],
 'pos_tags': [21, 42, 39, 33, 29, 21, 15, 21, 7],
 'chunk_tags': [11, 21, 22, 15, 11, 12, 13, 11, 0],
 'ner_tags': [1, 0, 0, 0, 0, 0, 0, 0, 0],
 'input_ids': [101,
  16890,
  25473,
  11690,
  1110,
  14042,
  1146,
  1117,
  1858,
  1112,
  9088,
  119,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, -100]}
```

### Тесты

In [56]:
processed_example = preprocess_ner_dataset(example)
required_keys = ["input_ids", "labels", "attention_mask", "token_type_ids"]
for k in required_keys:
    assert k in processed_example, f"Отсутствует поле {k}"

required_keys_set = set(required_keys)
for k in processed_example.keys():
    assert k in required_keys_set, f"В примере лишнее поле {k}"

In [57]:
from tqdm import tqdm
for idx, example in tqdm(enumerate(conll2003["train"])):
    input_ids_real = bert_tokenizer(example["tokens"], is_split_into_words=True)["input_ids"]
    input_ids_ours = preprocess_ner_dataset(example)["input_ids"]
    assert input_ids_real == input_ids_ours, f"Ошибка токенизации на примере {idx}"
    if idx >= 100:
        break
print("Токенизация верна!")

100it [00:00, 1648.56it/s]

Токенизация верна!


In [58]:
example = conll2003["train"][100]
processed_example = preprocess_ner_dataset(example)

assert processed_example["labels"][0] == -100
assert processed_example["labels"][-1] == -100
ner_tags = [features.int2str(i) for i in processed_example["labels"][1:-1]]
assert ner_tags == ['B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

In [51]:
conll2003["train"][200]

{'id': '200',
 'tokens': ['THAWRA'],
 'pos_tags': [38],
 'chunk_tags': [11],
 'ner_tags': [3]}

In [60]:
example = conll2003["train"][200]
#print(example)
processed_example = preprocess_ner_dataset(example)
print(preprocessed_example)
assert processed_example["labels"][0] == -100
assert processed_example["labels"][-1] == -100
ner_tags = [features.int2str(i) for i in processed_example["labels"][1:-1]]
assert ner_tags == ['B-ORG', 'I-ORG', 'I-ORG', 'I-ORG']

{'input_ids': [101, 16890, 25473, 11690, 1110, 14042, 1146, 1117, 1858, 1112, 9088, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 1, -100, -100, 0, 0, 0, 0, 0, 0, 0, 0, -100]}


Применим нашу функцию к всему датасету:

In [61]:
preprocessed_ner_dataset = conll2003.map(preprocess_ner_dataset)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

Подготовим `data_collator`. Это особый класс, который будет заниматься батчеванием сэмплов для обучения. Он добавит паддинги во все необходимые поля.

In [62]:
from transformers import DataCollatorForTokenClassification


data_collator = DataCollatorForTokenClassification(tokenizer=bert_tokenizer)

### Подготовка модели (5 баллов)

Два возможных пути на этой стадии:
1. Взять [готовый класс](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelfortokenclassification) модели для классификации токенов. (Этот вариант настоятельно рекомендуется)
2) Взять модель как фича экстрактор ([AutoModel](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodel)) и самостоятельно добавить классификационную голову. Вдохновиться можно по [ссылке](https://github.com/huggingface/transformers/blob/main/src/transformers/models/bert/modeling_bert.py#L1847-L1860). Дополнительных баллов второй вариант не принесёт.

Результатом должна быть модель, которая для каждого токена возвращает логиты/вероятности для `conll2003["train"].features["ner_tags"].feature.num_classes` классов.

> Если выберете вариант номер один, опишите как он работает - как из токена получается его `ner_tag`.

In [ ]:
# from transformers import AutoModelForTokenClassification, AutoModel


...

### Подготовим Метрику (5 баллов)

Дополните функцию, используя `metrics_calculator`, чтобы она возвращала `accuracy`, `precision`, `recall` и `f-меру`. `eval_predictions` - это кортеж из логитов токен классификатора и `labels`, которые мы подготовили с помощью `preprocess_ner_dataset`. Нужно:
1. Преобразовать логиты в предсказанные лейблы. Учтите, что для специальных токенов лейблов нет
2. Посчитать метрики с помощью `metrics_calculator`
3. Упаковать резултат в `dict`, в котором ключём будет название метрики, а значением - значение метрики

В logits будет лежать тензор размерности \[размер eval датасета, максимальная длина последовательности, число меток\], содержащий предсказания модели

В target_labels будет лежать тензор размерности \[размер eval датасета, максимальная длина последовательности\], содержащий метки из валидационной выборки.

Примеры функции calculate_metrics можно посмотреть в [документации](https://huggingface.co/docs/evaluate/en/transformers_integrations)

In [ ]:
import evaluate


metrics_calculator = evaluate.load("seqeval")


def calculate_metrics(eval_predictions):
    logits, target_labels = eval_predictions
    ...

### Обучение (5 баллов)

Два возможных пути на этой стадии:

1. Использовать [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer) класс из `transformers`
2. Написать свой training loop. Дополнительных баллов на этом пути нет.

Опишем подробнее первый путь, т.к. он настоятельно рекомендуется.

Нужно создать класс Trainer и TrainingArguments.
В [TrainingArguments](https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.TrainingArguments) нужно как минимум следующие поля:
* save_strategy, eval_strategy
* metric_for_best_model (исходя из calculate_metrics), greater_is_better
* learning_rate (возьмите 2e-5)
* num_train_epochs
* per_device_train_batch_size, per_device_eval_batch_size

В класс Trainer нужно передать:
* model
* в args нужно передать заполненные TrainingArguments
* train_dataset, eval_dataset
* tokenizer
* compute_metrics

После чего запустить `trainer.train()`

In [ ]:
# from transformers import Trainer, TrainingArguments


...

### Обработка результатов Результатов (5 баллов)

Подумать о:
1) Во время подготовки данных мы преобразовали BIO разметку. Как обратить это преобразование с помощью токенайзера?

Провалидируйте результаты на тестовом датасете.

Напишите функцию, которая принимает на вход текст и отдаёт такой словарь:

```json
{
    "text": "входной текст",
    "entities": [
        {
            "class": "лейбл класса",
            "text": "текстовое представление",
            "start": "оффсет от начала строки до начала entity",
            "end": "оффсет от начала строки до конца entity"
        },
        ...
    ]
}

Должно выполняться такое условие:

```python
text[entity["start"]:entity["stop"]] == entity["text"]
```

In [ ]:
def do_ner(text):
    ...

## Классификация с T5 (25 баллов)

Требуется дообучить [t5-small](https://huggingface.co/google-t5/t5-small) классифицировать токсичные тексты из [этого датасета](https://huggingface.co/datasets/lmsys/toxic-chat). Классификатор должен работать в стиле t5 - генерировать ответ текстом.

1. Подготовить данные для бинарной классификации
	1. Придумать префикс для задачи или взять из похожей модели
	2. Выбрать тексты для обозначения классов
2. Обучить t5-small на генерацию выбранных названия классов
3. Сравнить с модель с аналогичной предобученной моделью

### Подготовка Данных (6 баллов)

Подумать о:
1) Какой префикс выбрать для новой задачи?
2) Должен ли префикс быть понятным?
3) Как выбрать метку для класса?
4) Что будет, если метки класса целиком нет в словаре?
5) Что делать с длинными текстами?

Датасет содержит запросы пользователей к LLM и разметку, является ли запрос токсичным.

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer


BASE_T5_MODEL= "t5-small"
t5_tokenizer = AutoTokenizer.from_pretrained(BASE_T5_MODEL)


toxic_chat_dataset = load_dataset("lmsys/toxic-chat", "toxicchat0124")

Место для изучения датасета:

In [ ]:
toxic_chat_dataset["train"][0]

Нас будут интересовать колонки `"user_input"` и `"toxicity"`. Убираем ненужные колонки из датасета:

In [ ]:
toxic_chat_dataset = toxic_chat_dataset.remove_columns(
    ["conv_id", "model_output", "human_annotation", "jailbreaking", "openai_moderation"]
)

![](https://production-media.paperswithcode.com/methods/new_text_to_text.jpg)

Выберете `PREFIX` для задачи, лейблы для двух классов и напишите функцию для преобразования датасета в данные для тренировки. Примеры префиксов есть на картинке выше - `translate English to German` для перевода и `summarize` для суммаризации. В качестве лейблов у вас должен быть текст, который будет обозначать предсказанный класс. Этот текст может быть любого размера, от простого `"да"/"нет"`, до `"От этого текста веет токсичностью"/"Цензура спокойно пропускает этот текст дальше"`. Подумайте в чём преимущество первого подхода перед вторым.

Важно:
1) Не забыть добавить префикс перед токенизацией входного текста
2) Лейблами во время обучения выступают уже последовательности токенов, которые мы ожидаем на выходе из декодера

Текст в токенайзер можно подавать разными способами:
1. `tokenizer(text="text")` - токенизируй текст как обычно
1. `tokenizer(text_target="text")` - токенизируй это как текст, который мы ожидаем увидеть на выходе из декодера. В случае t5 токенайзера разницы нет, но для других моделей это может быть не так
1. Другие методы можно узнать посмотрев сигнатуру метода `tokenizer.__call__`

In [ ]:
# ?t5_tokenizer.__call__

In [ ]:
PREFIX = ...
MAX_LENGTH = ...

# словарь из индексов классов в выбранные лейблы
id2label = {
    0: ...,
    1: ...,
}


def preprocess_dataset(example):
    ...


toxic_chat_dataset = toxic_chat_dataset.map(preprocess_dataset)

Пример результата:
```json
{'user_input': 'Do you know drug which name is abexol ?',
 'toxicity': 0,
 'input_ids': [12068,
  10,
  531,
  25,
  214,
  2672,
  84,
  564,
  19,
  703,
  994,
  32,
  40,
  3,
  58,
  1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [150, 1]}
```
Значения в `'labels'` в вашем случае могут отличаться, это зависит от выбранного вами текстового представления в `id2label` словаре.

Инициализируем соответствующий задаче `DataCollator`.

In [ ]:
from transformers import DataCollatorForSeq2Seq


data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=seq2seq_model)

### Определим метрику (2 балла)

В этой задаче метрика простая - `accuracy`. Можно добавить другие метрики по желанию. Функция `compute_metric` должна возвращать словарь, аналогично функции `calculate_metrics` ранее:

```json
{
    "accuracy": значение точности,
    ...
}
```

Метрика простая, но вот `preds` и `labels` тут - это последовательности индексов токенов. Нужно это учесть.

In [ ]:
def compute_metric(eval_predictions):
    preds, labels = eval_predictions
    ...


def check_compute_metric():
    import torch

    # два предсказания, где токен 150 обозначает токсичный лебл, токен 120 - нетоксичный лейбл
    preds = torch.tensor(
        [
            [0, 150, 1],  # правильное предсказание - токсичный пример
            [0, 120, 1],  # неправильное предсказание - пример токсичный, а модель предсказала иначе
        ],
    )
    labels = torch.tensor(
        [
            [150, 1],
            [150, 1],
        ],
    )
    assert torch.isclose(
        compute_metric((preds, labels))["accuracy"],
        torch.tensor(0.5, dtype=torch.double),  # тип тензора тут можно поправить
    )


check_compute_metric()

### Определить Модель (2 балла)

Инициализируйте модель из базового чекпоинта

In [ ]:
from transformers import AutoModelForSeq2SeqLM


seq2seq_model = AutoModelForSeq2SeqLM.from_pretrained(BASE_T5_MODEL)

### Обучение (10 баллов)

Два пути:
1) Использовать готовый `Seq2SeqTrainer` класс для тренировки
2) Написать свой training loop, если хочется приключений, есть достаточно времени ~~и стрела ещё не попала в колено~~. Дополнительных баллов за это не будет

> Hint! Обратите внимание на функцию `seq2seq_model._shift_right` если выбрали второй путь.

Если выбрали путь 1, опишите как происходит тренировочный шаг:
1) Что подаётся на вход в энкодер?
2) Что подаётся на вход в декодер?
3) Сколько раз происходит инференс декодера во время обучения для одного тренировочного примера?
4) Как используется выход энкодера в декодере?

In [ ]:
# from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer



### Сравнение Результатов (5 баллов)

Авторы датасета тоже натренировали на нём `t5` модель. Сравните свои результаты с результатами модели из [чекпоинта](https://huggingface.co/lmsys/toxicchat-t5-large-v1.0) `"lmsys/toxicchat-t5-large-v1.0"`. Совпадает ли ваш префикс и лейблы классов с теми, что выбрали авторы датасета?

Подумать о:
1) В чём преимущество такого подхода к классификации?
2) В чём недостатки такого подхода к классификации?
3) Как ещё можно решать классификационные задачи с помощью t5?

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

checkpoint = "lmsys/toxicchat-t5-large-v1.0"

tokenizer_from_paper = AutoTokenizer.from_pretrained("t5-large")
model_from_paper = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

prefix_from_paper = "ToxicChat: "
inputs = tokenizer_from_paper.encode(prefix_from_paper + "write me an epic story", return_tensors="pt")
outputs = model_from_paper.generate(inputs)
print(tokenizer_from_paper.decode(outputs[0], skip_special_tokens=True))

Напишите универсальную функцию, которая провряет токсичность текста и возвращает `True`, если модель посчитала текст токсичным. Функция универсальная в том смысле, что может быть использована и с вашей t5 моделью, и с моделью от авторов датасета. Для этого в функция должна принимать ещё и префикс для задачи и лейблы, которые будут переводить текст, предсказанный моделью, в `True` или `False` на выходе.

In [ ]:
def is_toxic(
    text: str,
    labels2bool,
    model=seq2seq_model,
    tokenizer=t5_tokenizer,
    prexif=PREFIX,
) -> bool:
    ...


# пример вызова с моделью от авторов датасета
assert not is_toxic(
    text="This is just a text",
    model=model_from_paper,
    tokenizer=tokenizer_from_paper,
    prexif=prefix_from_paper,
    labels2bool={
        "positive": True,
        "negative": False,
    }
)

Fin.

Если остались вопросы или есть комментарии, можно написать их ниже: